In [1]:
# Import our dependencies
import os 
import datetime
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from matplotlib import pyplot as plt
import pandas as pd
from sklearn.cluster import KMeans
import numpy as np
import tensorflow as tf

### Preprocessing data

In [2]:
kickstarter_companies_df = pd.read_csv('Kickstarter_projects_Feb19.csv')
# new_companies_df = companies_df.drop(['Portfolio Exits'],axis = 1)
kickstarter_companies_df.head(10)

,id,name,currency,main_category,sub_category,launched_at,deadline,duration,goal_usd,city,state,country,blurb_length,name_length,status,start_month,end_month,start_Q,end_Q,usd_pledged
0,1687733153,Socks of Speed and Socks of Elvenkind,USD,games,Tabletop Games,2018-10-30 20:00:02,2018-11-15 17:59:00,16.0,2000.00000,Menasha,WI,US,14,7,successful,10,11,Q4,Q4,6061.00000
1,227936657,Power Punch Boot Camp: An All-Ages Graphic Novel,GBP,comics,Comic Books,2018-08-06 10:00:43,2018-09-05 10:00:43,30.0,3870.99771,Shepperton,England,GB,24,8,successful,8,9,Q3,Q3,3914.50512
2,454186436,"Live Printing with SX8: ""Squeegee Pulp Up""",USD,fashion,Apparel,2017-06-09 15:41:03,2017-07-09 15:41:03,30.0,1100.00000,Manhattan,NY,US,21,7,successful,6,7,Q2,Q3,1110.00000
3,629469071,Lost Dog Street Band's Next Album,USD,music,Country & Folk,2014-09-25 18:46:01,2014-11-10 06:00:00,45.0,3500.00000,Nashville,TN,US,15,6,successful,9,11,Q3,Q4,4807.00000
4,183973060,"Qto-X, a Tiny Lantern",USD,technology,Gadgets,2016-11-28 16:35:11,2017-01-27 16:35:11,60.0,30000.00000,Troy,MI,US,15,4,successful,11,1,Q4,Q1,40368.00000
5,122409435,"Aaron McDonnell EP ""Follow""",USD,music,Country & Folk,2014-01-22 18:21:37,2014-02-21 18:21:37,30.0,7500.00000,Seattle,WA,US,11,4,successful,1,2,Q1,Q1,7770.00000
6,421029848,Lush Life,USD,film & video,Drama,2018-08-17 18:58:18,2018-09-16 18:58:18,30.0,6000.00000,Los Angeles,CA,US,3,2,successful,8,9,Q3,Q3,6500.00000
7,1452339343,The biggest NASA fashion collection in the gal...,EUR,design,Graphic Design,2018-07-17 21:36:14,2018-08-16 21:36:14,30.0,1133.68788,Berlin,Berlin,DE,17,8,successful,7,8,Q3,Q3,1168.38400
8,815131323,The Nothing Box,USD,art,Sculpture,2011-09-17 17:02:56,2011-10-17 17:02:56,30.0,3000.00000,San Francisco,CA,US,19,3,successful,9,10,Q3,Q4,3511.00000
9,2132215273,"""Sittin' 'round waitin' for the world to end"" ...",USD,music,Country & Folk,2013-03-01 17:01:17,2013-04-30 16:01:17,60.0,1000.00000,Oceanside,CA,US,27,10,successful,3,4,Q1,Q2,1770.85000


In [3]:
kickstarter_companies_df["status"].value_counts()

successful    117307
failed         75241
Name: status, dtype: int64

In [4]:
kickstarter_companies_df['launched_at'] = pd.to_datetime(kickstarter_companies_df['launched_at'],infer_datetime_format=True).dt.strftime("%Y")
kickstarter_companies_df['deadline'] = pd.to_datetime(kickstarter_companies_df['deadline'],infer_datetime_format=True).dt.strftime("%Y")
kickstarter_companies_df.rename(columns={'launched_at':'Launch Year','deadline':'Deadline','main_category':'Industry','duration':'Duration(days)'},inplace=True)


,id,name,currency,Industry,sub_category,Launch Year,Deadline,Duration(days),goal_usd,city,state,country,blurb_length,name_length,status,start_month,end_month,start_Q,end_Q,usd_pledged
0,1687733153,Socks of Speed and Socks of Elvenkind,USD,games,Tabletop Games,2018,2018,16.0,2000.00000,Menasha,WI,US,14,7,successful,10,11,Q4,Q4,6061.00000
1,227936657,Power Punch Boot Camp: An All-Ages Graphic Novel,GBP,comics,Comic Books,2018,2018,30.0,3870.99771,Shepperton,England,GB,24,8,successful,8,9,Q3,Q3,3914.50512
2,454186436,"Live Printing with SX8: ""Squeegee Pulp Up""",USD,fashion,Apparel,2017,2017,30.0,1100.00000,Manhattan,NY,US,21,7,successful,6,7,Q2,Q3,1110.00000
3,629469071,Lost Dog Street Band's Next Album,USD,music,Country & Folk,2014,2014,45.0,3500.00000,Nashville,TN,US,15,6,successful,9,11,Q3,Q4,4807.00000
4,183973060,"Qto-X, a Tiny Lantern",USD,technology,Gadgets,2016,2017,60.0,30000.00000,Troy,MI,US,15,4,successful,11,1,Q4,Q1,40368.00000


In [5]:
kickstarter_companies_df.head(10)

,id,name,currency,Industry,sub_category,Launch Year,Deadline,Duration(days),goal_usd,city,state,country,blurb_length,name_length,status,start_month,end_month,start_Q,end_Q,usd_pledged
0,1687733153,Socks of Speed and Socks of Elvenkind,USD,games,Tabletop Games,2018,2018,16.0,2000.00000,Menasha,WI,US,14,7,successful,10,11,Q4,Q4,6061.00000
1,227936657,Power Punch Boot Camp: An All-Ages Graphic Novel,GBP,comics,Comic Books,2018,2018,30.0,3870.99771,Shepperton,England,GB,24,8,successful,8,9,Q3,Q3,3914.50512
2,454186436,"Live Printing with SX8: ""Squeegee Pulp Up""",USD,fashion,Apparel,2017,2017,30.0,1100.00000,Manhattan,NY,US,21,7,successful,6,7,Q2,Q3,1110.00000
3,629469071,Lost Dog Street Band's Next Album,USD,music,Country & Folk,2014,2014,45.0,3500.00000,Nashville,TN,US,15,6,successful,9,11,Q3,Q4,4807.00000
4,183973060,"Qto-X, a Tiny Lantern",USD,technology,Gadgets,2016,2017,60.0,30000.00000,Troy,MI,US,15,4,successful,11,1,Q4,Q1,40368.00000
5,122409435,"Aaron McDonnell EP ""Follow""",USD,music,Country & Folk,2014,2014,30.0,7500.00000,Seattle,WA,US,11,4,successful,1,2,Q1,Q1,7770.00000
6,421029848,Lush Life,USD,film & video,Drama,2018,2018,30.0,6000.00000,Los Angeles,CA,US,3,2,successful,8,9,Q3,Q3,6500.00000
7,1452339343,The biggest NASA fashion collection in the gal...,EUR,design,Graphic Design,2018,2018,30.0,1133.68788,Berlin,Berlin,DE,17,8,successful,7,8,Q3,Q3,1168.38400
8,815131323,The Nothing Box,USD,art,Sculpture,2011,2011,30.0,3000.00000,San Francisco,CA,US,19,3,successful,9,10,Q3,Q4,3511.00000
9,2132215273,"""Sittin' 'round waitin' for the world to end"" ...",USD,music,Country & Folk,2013,2013,60.0,1000.00000,Oceanside,CA,US,27,10,successful,3,4,Q1,Q2,1770.85000


In [8]:
kickstarter_companies_df['Duration(days)'].value_counts()

30.0    86776
60.0    14363
45.0     7505
31.0     6861
35.0     4980
        ...  
79.0        4
73.0        3
93.0        2
92.0        2
83.0        1
Name: Duration(days), Length: 93, dtype: int64

In [11]:
kickstarter_companies_df.dtypes

id                  int64
name               object
currency           object
Industry           object
sub_category       object
Launch Year        object
Deadline           object
Duration(days)    float64
goal_usd          float64
city               object
state              object
country            object
blurb_length        int64
name_length         int64
status             object
start_month         int64
end_month           int64
start_Q            object
end_Q              object
usd_pledged       float64
dtype: object

In [13]:
kickstarter_companies_df['usd_pledged'].mean()

13711.658643570801